In [1]:
import sys
import logging
import argparse
import configparser
import os
import shutil
import torch
import gym
import git
from crowd_sim.envs.utils.robot import Robot
from crowd_nav.utils.trainer import Trainer
from crowd_nav.utils.memory import ReplayMemory
from crowd_nav.utils.explorer import Explorer
from crowd_nav.policy.policy_factory import policy_factory

/home/or06360/miniconda3/envs/crowdnav/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parsing Arguments 

``python train.py --policy MambaRL --output_dir data/output/checking_mamba_onvscode --gpu``

In [2]:
gpu = True
debug = True
policy = 'MambaRL'
output_path = 'data/output/training_notebook/'

In [3]:
print('ayee')
parser = argparse.ArgumentParser('Parse configuration file')
print('ranoo')
parser.add_argument('--env_config', type=str, default='configs/env.config')
parser.add_argument('--policy', type=str, default=policy)
parser.add_argument('--policy_config', type=str, default='configs/policy.config')
parser.add_argument('--train_config', type=str, default='configs/train.config')
parser.add_argument('--output_dir', type=str, default=output_path)
parser.add_argument('--weights', type=str)
parser.add_argument('--resume', default=False, action='store_true')
parser.add_argument('--gpu', default=gpu, action='store_true')
parser.add_argument('--debug', default=debug, action='store_true')
print('baby')
args = parser.parse_args(args = [])

ayee
ranoo
baby


# Configure Paths

In [4]:
make_new_dir = True
if os.path.exists(args.output_dir):
    key = input('Output directory already exists! Overwrite the folder? (y/n)')
    if key == 'y' and not args.resume:
        shutil.rmtree(args.output_dir)
    else:
        make_new_dir = False
        args.env_config = os.path.join(args.output_dir, os.path.basename(args.env_config))
        args.policy_config = os.path.join(args.output_dir, os.path.basename(args.policy_config))
        args.train_config = os.path.join(args.output_dir, os.path.basename(args.train_config))
if make_new_dir:
    os.makedirs(args.output_dir)
    shutil.copy(args.env_config, args.output_dir)
    shutil.copy(args.policy_config, args.output_dir)
    shutil.copy(args.train_config, args.output_dir)
log_file = os.path.join(args.output_dir, 'output.log')
il_weight_file = os.path.join(args.output_dir, 'il_model.pth')
rl_weight_file = os.path.join(args.output_dir, 'rl_model.pth')


# Configure Logging

In [5]:
mode = 'a' if args.resume else 'w'
file_handler = logging.FileHandler(log_file, mode=mode)
stdout_handler = logging.StreamHandler(sys.stdout)
level = logging.INFO if not args.debug else logging.DEBUG
logging.basicConfig(level=level, handlers=[stdout_handler, file_handler],
                    format='%(asctime)s, %(levelname)s: %(message)s', datefmt="%Y-%m-%d %H:%M:%S")
repo = git.Repo(search_parent_directories=True)
logging.info('Current git head hash code: %s'.format(repo.head.object.hexsha))
device = torch.device("cuda:0" if torch.cuda.is_available() and args.gpu else "cpu")
logging.info('Using device: %s', device)


2024-02-24 14:43:23, DEBUG: Popen(['git', 'cat-file', '--batch-check'], cwd=/home/or06360/Capstone/CrowdNav-tests, stdin=<valid stream>, shell=False, universal_newlines=False)
2024-02-24 14:43:23, INFO: Current git head hash code: %s
2024-02-24 14:43:23, INFO: Using device: cuda:0


# configure policy

In [6]:

policy = policy_factory[args.policy]()
print(policy)
if not policy.trainable:
    parser.error('Policy has to be trainable')
if args.policy_config is None:
    parser.error('Policy config has to be specified for a trainable network')
policy_config = configparser.RawConfigParser()
policy_config.read(args.policy_config)
policy.configure(policy_config)
policy.set_device(device)

input dimension = 13
ValueNetork(
  (mambaLayer): Mamba(
    (in_proj): Linear(in_features=13, out_features=52, bias=False)
    (conv1d): Conv1d(26, 26, kernel_size=(4,), stride=(1,), padding=(3,), groups=26)
    (act): SiLU()
    (x_proj): Linear(in_features=26, out_features=33, bias=False)
    (dt_proj): Linear(in_features=1, out_features=26, bias=True)
    (out_proj): Linear(in_features=26, out_features=13, bias=False)
  )
  (value_network): Sequential(
    (0): Linear(in_features=13, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=1, bias=True)
  )
)
2024-02-24 14:43:25, INFO: Policy:MambaRL


# configure environment

In [7]:
env_config = configparser.RawConfigParser()
env_config.read(args.env_config)
env = gym.make('CrowdSim-v0')
env.configure(env_config)
robot = Robot(env_config, 'robot')
env.set_robot(robot)


2024-02-24 14:43:28, INFO: human number: 5
2024-02-24 14:43:28, INFO: Not randomize human's radius and preferred speed
2024-02-24 14:43:28, INFO: Training simulation: circle_crossing, test simulation: circle_crossing
2024-02-24 14:43:28, INFO: Square width: 10.0, circle width: 4.0


 # read training parameters

In [8]:
if args.train_config is None:
    parser.error('Train config has to be specified for a trainable network')
train_config = configparser.RawConfigParser()
train_config.read(args.train_config)
rl_learning_rate = train_config.getfloat('train', 'rl_learning_rate')
train_batches = train_config.getint('train', 'train_batches')
train_episodes = train_config.getint('train', 'train_episodes')
sample_episodes = train_config.getint('train', 'sample_episodes')
target_update_interval = train_config.getint('train', 'target_update_interval')
evaluation_interval = train_config.getint('train', 'evaluation_interval')
capacity = train_config.getint('train', 'capacity')
epsilon_start = train_config.getfloat('train', 'epsilon_start')
epsilon_end = train_config.getfloat('train', 'epsilon_end')
epsilon_decay = train_config.getfloat('train', 'epsilon_decay')
checkpoint_interval = train_config.getint('train', 'checkpoint_interval')

# configure trainer and explorer

In [9]:
memory = ReplayMemory(capacity)
print(policy)
model = policy.get_model()
batch_size = train_config.getint('trainer', 'batch_size')
trainer = Trainer(model, memory, device, batch_size)
explorer = Explorer(env, robot, device, memory, policy.gamma, target_policy=policy)

# imitation learning

In [10]:

if args.resume:
    if not os.path.exists(rl_weight_file):
        logging.error('RL weights does not exist')
    model.load_state_dict(torch.load(rl_weight_file))
    rl_weight_file = os.path.join(args.output_dir, 'resumed_rl_model.pth')
    logging.info('Load reinforcement learning trained weights. Resume training')
elif os.path.exists(il_weight_file):
    model.load_state_dict(torch.load(il_weight_file))
    logging.info('Load imitation learning trained weights.')
else:
    il_episodes = train_config.getint('imitation_learning', 'il_episodes')
    il_policy = train_config.get('imitation_learning', 'il_policy')
    il_epochs = train_config.getint('imitation_learning', 'il_epochs')
    il_learning_rate = train_config.getfloat('imitation_learning', 'il_learning_rate')
    trainer.set_learning_rate(il_learning_rate)
    if robot.visible:
        safety_space = 0
    else:
        safety_space = train_config.getfloat('imitation_learning', 'safety_space')
    il_policy = policy_factory[il_policy]()
    il_policy.multiagent_training = policy.multiagent_training
    il_policy.safety_space = safety_space
    robot.set_policy(il_policy)
    explorer.run_k_episodes(il_episodes, 'train', update_memory=True, imitation_learning=True)
    trainer.optimize_epoch(il_epochs)
    torch.save(model.state_dict(), il_weight_file)
    logging.info('Finish imitation learning. Weights saved.')
    logging.info('Experience set size: %d/%d', len(memory), memory.capacity)
explorer.update_target_model(model)
    

2024-02-24 14:43:35, INFO: Current learning rate: 0.010000
Running for 3000 episodes


100%|██████████| 3000/3000 [27:58<00:00,  1.79it/s]

2024-02-24 15:11:34, INFO: TRAIN has success rate: 0.89, collision rate: 0.09, nav time: 12.18, total reward: 0.2416


I have changed the code a little bit, i.e. right now the code olbly works for training for first time

torch.Size([100, 5, 13])
torch.Size([100, 5, 13])
torch.Size([100, 5, 1])
Prints in trainer.py
torch.Size([100, 5, 1])
torch.Size([100, 1])


/home/or06360/miniconda3/envs/crowdnav/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([100, 5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (5) must match the size of tensor b (100) at non-singleton dimension 1